In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download("wordnet")
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
%matplotlib inline
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anfang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/anfang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
dataset = pd.read_csv('dataset.csv', index_col = 'Unnamed: 0')
print(dataset.shape)

(24783, 6)


In [3]:
dataset.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
label =list(dataset['class'].values)
text = list(dataset['tweet'].values)

In [5]:
print(len(label))
print(len(text))

24783
24783


In [7]:
file2 = open("data/input_data_2.txt", "a")
file3 = open("data/input_data_3.txt", "a")
for i in range(len(label)):
    write_in = str(label[i]) + "\t" + text[i] + "\n"
    if label[i] == 2:
        file2.write(write_in)
    if label[i] == 0:
        file3.write(write_in)
file2.close
file3.close

<function TextIOWrapper.close()>

In [8]:
dataset['class'].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [11]:
# do basic data preprocess, then generate new data
import string
def data_preprocess(review):
    review = re.sub(r'^\s+|\s+?$','',review) # remove leading and trailing whitespace
    review = re.sub(r'@[\w\-]+', '', review) # remove mentions
    review = re.sub(r'RT[\s]+', '', review) #remove retweet text 'RT'
    review = re.sub('[^a-zA-Z]',' ',review)#replace non-character with space
    review = review.lower() #lower the text
    review = re.sub('\[.*?\]', '', review)
    review = re.sub('https?://\S+|www\.\S+', '', review) # remove hyperlinks
    review = re.sub(r'#', '', review) # remove hashtags
    review = re.sub('[%s]' % re.escape(string.punctuation), '', review) # remove escape characters
    review = re.sub('\n', '', review)
    review = re.sub('\w*\d\w*', '', review)
    return review

In [12]:
file2_2 = open("data/input_data_2_2.txt", "a")
file3_2 = open("data/input_data_3_2.txt", "a")
for i in range(len(label)):
    new_text = data_preprocess(text[i])
    write_in = str(label[i]) + "\t" + new_text + "\n"
    if label[i] == 2:
        file2_2.write(write_in)
    if label[i] == 0:
        file3_2.write(write_in)
file2_2.close
file3_2.close

<function TextIOWrapper.close()>

command line as follows:<br>
python code/augment.py --input=data/input_data_2.txt --output=2_0.txt --num_aug=5<br>
python code/augment.py --input=data/input_data_2_2.txt --output=2_1.txt --num_aug=5<br>
python code/augment.py --input=data/input_data_3.txt --output=3_0.txt --num_aug=14<br>
python code/augment.py --input=data/input_data_3_2.txt --output=3_1.txt --num_aug=14<br>